## Esta primeira análise será feita se baseando no modelo lógico 'Análise Obesidade', o qual foi criado juntando os datasets ObesityStats e HeartDisease.

In [1]:
#IMPORTS

!pip3 install pandas
import sqlite3 as sql
import pandas as pd
import io
import os

In [2]:
# Limpar o buffer
try:
  os.remove("HeartDisease.csv")
  os.remove("ObesityStats.csv")
  os.remove("HeartDisease.csv.1")
  os.remove("ObesityStats.csv.1")
  os.remove("project.db") 
except OSError:      
  pass


## Download dos datasets já filtrados pela equipe e postos no repositório da equipe no GitHub.

In [3]:
!wget https://raw.githubusercontent.com/MatheusCod/Kinda_SUS-MC536_2s2020/main/stage03/CSVs/ObesityStats.csv
!wget https://raw.githubusercontent.com/MatheusCod/Kinda_SUS-MC536_2s2020/main/stage03/CSVs/HeartDisease.csv

--2020-11-13 04:17:36--  https://raw.githubusercontent.com/MatheusCod/Kinda_SUS-MC536_2s2020/main/stage03/CSVs/ObesityStats.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2716335 (2.6M) [text/plain]
Saving to: ‘ObesityStats.csv’

ObesityStats.csv    100%[===================>]   2.59M  --.-KB/s    in 0.1s    

2020-11-13 04:17:36 (26.3 MB/s) - ‘ObesityStats.csv’ saved [2716335/2716335]

--2020-11-13 04:17:36--  https://raw.githubusercontent.com/MatheusCod/Kinda_SUS-MC536_2s2020/main/stage03/CSVs/HeartDisease.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting res

## Ligação dos datasets com a biblioteca sqlite3 do Python.

In [4]:
HeartDisease=pd.read_csv('HeartDisease.csv')
ObesityStats=pd.read_csv('ObesityStats.csv')

In [5]:
conn = sql.connect('project.db')

c = conn.cursor()

c.execute("DROP TABLE IF EXISTS ObesityStats")
c.execute("DROP TABLE IF EXISTS HeartDisease")

ObesityStats.to_sql('ObesityStats', conn)
HeartDisease.to_sql('HeartDisease', conn)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [6]:
AvgObesityData = pd.read_sql("""
  SELECT LocationAbbr, Question, Gender, AVG(Data_Value) Average
  FROM ObesityStats
  WHERE Question = 'Percent of adults aged 18 years and older who have obesity'
  GROUP BY LocationAbbr, Gender
  ORDER BY LocationAbbr
""", conn)

HeartData = pd.read_sql("""
  SELECT LocationAbbr, Topic, Stratification1, Data_Value, Data_Value_Unit
  FROM HeartDisease
  WHERE (Stratification1 = 'Male' OR Stratification1 = 'Female') AND Stratification2 = 'Overall' AND Topic = 'Heart Disease Mortality'
  GROUP BY LocationAbbr, Stratification1
  ORDER BY LocationAbbr
""", conn)

c.execute("DROP TABLE IF EXISTS AvgObesityData")
c.execute("DROP TABLE IF EXISTS HeartData")

AvgObesityData = AvgObesityData.dropna()
HeartData = HeartData.dropna()

AvgObesityData.to_sql('AvgObesityData', conn)
HeartData.to_sql('HeartData', conn)


## A criação da query que representa o modelo lógico 'Análise Obesidade' é a Query 1, logo após a Quary 2 e 3.

## Query 2: Médias de Obesidade e Morte por doenças do coração por Estado para o sexo masculino.

In [7]:
query2 = pd.read_sql(""" 
  SELECT HD.LocationAbbr, OD.Gender, OD.Question, OD.Average, HD.Topic, HD.Data_Value, HD.Data_Value_Unit
  FROM HeartData HD, AvgObesityData OD
  WHERE HD.LocationAbbr = OD.LocationAbbr AND HD.Stratification1 = 'Male' AND OD.Gender = 'Male'
  ORDER BY HD.LocationAbbr
""", conn)
c.execute("DROP TABLE IF EXISTS query2")
query2.to_sql('query2', conn)
query2

,LocationAbbr,Gender,Question,Average,Topic,Data_Value,Data_Value_Unit
0,AK,Male,Percent of adults aged 18 years and older who ...,29.2,Heart Disease Mortality,350.9,"per 100,000 population"
1,AL,Male,Percent of adults aged 18 years and older who ...,30.9,Heart Disease Mortality,540.9,"per 100,000 population"
2,AR,Male,Percent of adults aged 18 years and older who ...,35.2,Heart Disease Mortality,518.5,"per 100,000 population"
3,AZ,Male,Percent of adults aged 18 years and older who ...,26.5,Heart Disease Mortality,344.9,"per 100,000 population"
4,CA,Male,Percent of adults aged 18 years and older who ...,25.1,Heart Disease Mortality,499.9,"per 100,000 population"
5,CO,Male,Percent of adults aged 18 years and older who ...,20.8,Heart Disease Mortality,309.4,"per 100,000 population"
6,CT,Male,Percent of adults aged 18 years and older who ...,25.5,Heart Disease Mortality,369.4,"per 100,000 population"
7,DE,Male,Percent of adults aged 18 years and older who ...,30.6,Heart Disease Mortality,401.8,"per 100,000 population"
8,FL,Male,Percent of adults aged 18 years and older who ...,27.5,Heart Disease Mortality,373.0,"per 100,000 population"
9,GA,Male,Percent of adults aged 18 years and older who ...,29.2,Heart Disease Mortality,434.3,"per 100,000 population"


## Query 3: Médias de Obesidade e Morte por doenças do coração por Estado para o sexo feminino.

In [8]:
query3 = pd.read_sql(""" 
  SELECT HD.LocationAbbr, OD.Gender, OD.Question, OD.Average, HD.Topic, HD.Data_Value, HD.Data_Value_Unit
  FROM HeartData HD, AvgObesityData OD
  WHERE HD.LocationAbbr = OD.LocationAbbr AND HD.Stratification1 = 'Female' AND OD.Gender = 'Female'
  ORDER BY HD.LocationAbbr
""", conn)
c.execute("DROP TABLE IF EXISTS query3")
query3.to_sql('query3', conn)
query3

,LocationAbbr,Gender,Question,Average,Topic,Data_Value,Data_Value_Unit
0,AK,Female,Percent of adults aged 18 years and older who ...,27.5,Heart Disease Mortality,196.7,"per 100,000 population"
1,AL,Female,Percent of adults aged 18 years and older who ...,34.0,Heart Disease Mortality,352.6,"per 100,000 population"
2,AR,Female,Percent of adults aged 18 years and older who ...,34.1,Heart Disease Mortality,334.1,"per 100,000 population"
3,AZ,Female,Percent of adults aged 18 years and older who ...,27.1,Heart Disease Mortality,211.5,"per 100,000 population"
4,CA,Female,Percent of adults aged 18 years and older who ...,23.1,Heart Disease Mortality,227.5,"per 100,000 population"
5,CO,Female,Percent of adults aged 18 years and older who ...,21.8,Heart Disease Mortality,197.1,"per 100,000 population"
6,CT,Female,Percent of adults aged 18 years and older who ...,24.4,Heart Disease Mortality,230.8,"per 100,000 population"
7,DE,Female,Percent of adults aged 18 years and older who ...,31.5,Heart Disease Mortality,265.8,"per 100,000 population"
8,FL,Female,Percent of adults aged 18 years and older who ...,25.3,Heart Disease Mortality,227.0,"per 100,000 population"
9,GA,Female,Percent of adults aged 18 years and older who ...,31.5,Heart Disease Mortality,355.5,"per 100,000 population"


In [11]:
query1 = pd.read_sql(""" 
  SELECT query2.LocationAbbr, query2.Data_Value Mortalidade_homens, query3.Data_Value Mortalidade_mulheres, query2.Average Obesidade_homens, query3.Average Obesidade_mulheres
  FROM query2, query3
  WHERE query2.LocationAbbr = query3.LocationAbbr
  ORDER BY query2.Data_Value
""", conn)
query1.to_sql('query1', conn)
query1

,LocationAbbr,Mortalidade_homens,Mortalidade_mulheres,Obesidade_homens,Obesidade_mulheres
0,MN,301.3,173.1,27.0,23.8
1,CO,309.4,197.1,20.8,21.8
2,OR,330.0,199.2,28.0,25.0
3,UT,338.0,241.8,24.6,23.7
4,AZ,344.9,211.5,26.5,27.1
5,MA,350.4,211.8,25.9,21.4
6,AK,350.9,196.7,29.2,27.5
7,NM,352.0,223.6,25.6,27.1
8,HI,354.3,190.2,24.7,18.7
9,NH,358.2,229.7,28.7,24.8


## Query 4: Estados em que o índice de Obesidade é maior do que a média e Morte por doenças do coração dos sexos feminino e masculino.

In [12]:
query4 = pd.read_sql(""" 
  SELECT query2.LocationAbbr, query2.Average Obesidade_homens, query3.Average Obesidade_mulheres, query2.Data_Value Mortalidade_homens, query3.Data_Value Mortalidade_mulheres
  FROM query2, query3
  WHERE query2.LocationAbbr = query3.LocationAbbr AND query2.Average > query3.Average
  ORDER BY query2.LocationAbbr
""", conn)
query4

,LocationAbbr,Obesidade_homens,Obesidade_mulheres,Mortalidade_homens,Mortalidade_mulheres
0,AK,29.2,27.5,350.9,196.7
1,AR,35.2,34.1,518.5,334.1
2,CA,25.1,23.1,499.9,227.5
3,CT,25.5,24.4,369.4,230.8
4,FL,27.5,25.3,373.0,227.0
5,HI,24.7,18.7,354.3,190.2
6,IA,32.4,30.1,410.4,251.7
7,ID,31.2,27.9,363.1,260.6
8,KS,30.5,29.4,391.2,236.5
9,KY,33.3,33.1,500.1,310.4


## Query 5: Média da Obesidade e Morte por doenças do coração dos sexos feminino e masculino.

In [13]:
query5 = pd.read_sql(""" 
  SELECT AVG(query2.Average) Obesidade_homens, AVG(query3.Average) Obesidade_mulheres, AVG(query2.Data_Value) Mortalidade_homens, AVG(query3.Data_Value) Mortalidade_mulheres
  FROM query2, query3
  GROUP BY query2.Gender, query3.Gender
""", conn)
c.execute("DROP TABLE IF EXISTS query5")
query5.to_sql('query5', conn)
query5

,Obesidade_homens,Obesidade_mulheres,Mortalidade_homens,Mortalidade_mulheres
0,29.043137,28.458824,416.398039,259.943137


## Query 6: Estados onde as Médias da Obesidade é maior do que a Média Geral.

In [14]:
query6 = pd.read_sql(""" 
  SELECT query2.LocationAbbr, query2.Average Obesidade_homens, query3.Average Obesidade_mulheres, query2.Data_Value Mortalidade_homens, query3.Data_Value Mortalidade_mulheres
  FROM query2, query3, query5
  WHERE query2.LocationAbbr = query3.LocationAbbr AND query2.Average > query5.Obesidade_homens AND query3.Average > query5.Obesidade_mulheres
  ORDER BY query2.LocationAbbr
""", conn)
c.execute("DROP TABLE IF EXISTS query6")
query6.to_sql('query6', conn)
query6

,LocationAbbr,Obesidade_homens,Obesidade_mulheres,Mortalidade_homens,Mortalidade_mulheres
0,AL,30.9,34.0,540.9,352.6
1,AR,35.2,34.1,518.5,334.1
2,DE,30.6,31.5,401.8,265.8
3,GA,29.2,31.5,434.3,355.5
4,IA,32.4,30.1,410.4,251.7
5,IN,31.1,32.6,456.5,284.6
6,KS,30.5,29.4,391.2,236.5
7,KY,33.3,33.1,500.1,310.4
8,LA,31.1,35.0,522.2,330.4
9,ME,29.1,28.7,363.6,227.5


## Query 7: Estados onde a Mortalidade por doenças do coração é maior do que a média geral.

In [15]:
query7 = pd.read_sql(""" 
  SELECT query2.LocationAbbr, query2.Average Obesidade_homens, query3.Average Obesidade_mulheres, query2.Data_Value Mortalidade_homens, query3.Data_Value Mortalidade_mulheres
  FROM query2, query3, query5
  WHERE query2.LocationAbbr = query3.LocationAbbr AND query2.Data_Value > query5.Mortalidade_homens AND query3.Data_Value > query5.Mortalidade_mulheres
  ORDER BY query2.LocationAbbr
""", conn)
c.execute("DROP TABLE IF EXISTS query7")
query7.to_sql('query7', conn)
query7

,LocationAbbr,Obesidade_homens,Obesidade_mulheres,Mortalidade_homens,Mortalidade_mulheres
0,AL,30.9,34.0,540.9,352.6
1,AR,35.2,34.1,518.5,334.1
2,GA,29.2,31.5,434.3,355.5
3,IL,28.3,30.5,423.3,261.0
4,IN,31.1,32.6,456.5,284.6
5,KY,33.3,33.1,500.1,310.4
6,LA,31.1,35.0,522.2,330.4
7,MD,27.8,28.8,420.8,264.7
8,MI,31.1,31.9,482.1,311.3
9,MO,30.7,30.2,473.3,298.7


## Query 8: Estados em que as Médias de Obesidade e Mortalidade são maiores do que as médias gerais.

In [16]:
query8 = pd.read_sql(""" 
  SELECT query6.LocationAbbr, query6.Obesidade_homens, query6.Obesidade_mulheres, query6.Mortalidade_homens, query6.Mortalidade_mulheres
  FROM query6, query7
  WHERE query6.LocationAbbr = query7.LocationAbbr 
  ORDER BY query6.LocationAbbr
""", conn)
c.execute("DROP TABLE IF EXISTS query8")
query8.to_sql('query8', conn)
query8

,LocationAbbr,Obesidade_homens,Obesidade_mulheres,Mortalidade_homens,Mortalidade_mulheres
0,AL,30.9,34.0,540.9,352.6
1,AR,35.2,34.1,518.5,334.1
2,GA,29.2,31.5,434.3,355.5
3,IN,31.1,32.6,456.5,284.6
4,KY,33.3,33.1,500.1,310.4
5,LA,31.1,35.0,522.2,330.4
6,MI,31.1,31.9,482.1,311.3
7,MO,30.7,30.2,473.3,298.7
8,MS,33.8,36.4,564.3,361.9
9,OH,30.0,30.8,459.2,288.4
